In [1]:

from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [5]:

parser = argparse.ArgumentParser(description="VAE MNIST EXAMPLE")

In [6]:
parser.add_argument('--batch-size2', type=int, default=128, metavar='N',
help='input batch size for training (default: 128)')

_StoreAction(option_strings=['--batch-size2'], dest='batch_size2', nargs=None, const=None, default=128, type=<class 'int'>, choices=None, help='input batch size for training (default: 128)', metavar='N')

In [7]:
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
help='how many batches to wait before logging training status')


_StoreAction(option_strings=['--log-interval'], dest='log_interval', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='how many batches to wait before logging training status', metavar='N')

In [8]:

args = parser.parse_args(args=['--batch-size2','128','--epochs','10'])

In [9]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [10]:
torch.manual_seed(args.seed)

device = torch.device("cuda" if args.cuda else "cpu")

kwargs = {'num_workers':1, 'pin_memory':True} if args.cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=True,download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size2,shuffle=True,**kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=False,transform=transforms.ToTensor()),
    batch_size=args.batch_size2,shuffle=True,**kwargs)

In [19]:
args.batch_size2

128

In [13]:

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(784,400)
        self.fc21 = nn.Linear(400,20)
        self.fc22 = nn.Linear(400,20)
        self.fc3 = nn.Linear(20,400)
        self.fc4 = nn.Linear(400,784)
        
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    #入力から中心と分散の対数を作っている
    
    def reparameterize(self,mu,logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        #torch.randn_like(std)はstdと同じ次元の正規乱数を与えている。N(0,1)**size(std)
        return mu + eps*std
    #zの値を確率的に作っている。
    
    def decode(self,z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))
    #zからNNを通したあとに、sigmoidで押し込んでxの値を作っている
    
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1,784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z),mu,logvar
    #出力はxの予測値とzを算出するときの中心と分散になっている
    
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-3)



In [14]:
def loss_function(recon_x,x,mu,logvar):
    BCE = F.binary_cross_entropy(recon_x,x.view(-1,784),reduction='sum')
    #recon_xはsigmoidをdecordeで通されているので、[0,1]になっている。
    KLD = -0.5*torch.sum(1+logvar-mu.pow(2)-logvar.exp())
    
    return BCE+KLD

In [15]:

def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data,_) in enumerate(train_loader):
        #ラベルは使わないようだ
        data = data.to(device)
        #データをGPUにおくる
        optimizer.zero_grad()
        #傾きの初期化
        recon_batch, mu, logvar = model(data)
        #VAEからバッヂ分のデータと中心と分散のログを受け取る
        loss = loss_function(recon_batch,data,mu,logvar)
        #バッヂ分のロスを計算。バッヂ数で割っていないようだ。書き出しのときにデータ数で割っている。
        loss.backward()
        #傾きを計算
        train_loss+= loss.item()
        optimizer.step()
        #パラメータの更新
        if batch_idx % args.log_interval ==0:
            #一定間隔でロスを書き出し
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch,batch_idx*len(data),len(train_loader.dataset),
                100.*batch_idx/len(train_loader),
                loss.item()/len(data)
            ))

In [25]:

def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():#バックプロパゲーションを行わないから、微分情報を残さない
        for i, (data, _) in enumerate(test_loader):
            #テストデータを取り出す。ラベルはいらない。
            data =data.to(device)
            #データをGPUに送る
            recon_batch, mu, logvar = model(data)
            #データからバッヂ分の再現画像と中心と分散の対数を出している。
            test_loss += loss_function(recon_batch,data,mu,logvar).item()
            #テストデータでのロスを出している
            if i == 0:
                #はじめのバッヂについて
                n = min(data.size(0),16)
                #nをバッヂのデータ数か８との小さい方として
                comparizon = torch.cat([data[:n],recon_batch.view(args.batch_size2,1,28,28)[:n]])
                #元データと再現データを並べる
                save_image(comparizon.cpu(),'./results/reconstruction_' + str(epoch) + '.png',nrow=n)
                
                test_loss /= len(test_loader.dataset)
                print('====> Test set loss: {:.4f}'.format(test_loss))

In [30]:
args.epochs=20
args.epochs

20

In [31]:
for epoch in range(1, args.epochs +1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(81,20).to(device)
        #20次元の潜在空間のデータを64個作成して、GPUに送る
        sample = model.decode(sample).cpu()
        #ランダムな潜在変数から画像を生成する。
        save_image(sample.view(81,1,28,28),'results/sample_' + str(epoch) + '.png',nrow=9)
        #生成した６４個のデータを記録しておく

Train Epoch: 1 [0/60000 (0%)]	Loss: 101.927040
Train Epoch: 1 [1280/60000 (2%)]	Loss: 102.578384
Train Epoch: 1 [2560/60000 (4%)]	Loss: 101.506149
Train Epoch: 1 [3840/60000 (6%)]	Loss: 98.650948
Train Epoch: 1 [5120/60000 (9%)]	Loss: 97.793457
Train Epoch: 1 [6400/60000 (11%)]	Loss: 103.184891
Train Epoch: 1 [7680/60000 (13%)]	Loss: 99.731468
Train Epoch: 1 [8960/60000 (15%)]	Loss: 101.639252
Train Epoch: 1 [10240/60000 (17%)]	Loss: 100.950104
Train Epoch: 1 [11520/60000 (19%)]	Loss: 99.981804
Train Epoch: 1 [12800/60000 (21%)]	Loss: 102.596817
Train Epoch: 1 [14080/60000 (23%)]	Loss: 101.299881
Train Epoch: 1 [15360/60000 (26%)]	Loss: 94.809441
Train Epoch: 1 [16640/60000 (28%)]	Loss: 102.212860
Train Epoch: 1 [17920/60000 (30%)]	Loss: 102.460861
Train Epoch: 1 [19200/60000 (32%)]	Loss: 101.338280
Train Epoch: 1 [20480/60000 (34%)]	Loss: 104.402313
Train Epoch: 1 [21760/60000 (36%)]	Loss: 102.336838
Train Epoch: 1 [23040/60000 (38%)]	Loss: 101.812355
Train Epoch: 1 [24320/60000 (41%)